# Ciclos Políticos do Ceará

**Trabalho de Conclusão de Curso — MBA em Jornalismo de Dados (IDP)** 
**Responsável:** Igor Cavalcante — victorigor.ac@gmail.com  
**Tema:** Análise dos ciclos políticos do Ceará ao longo de 30 anos  
**Etapa:** 2/3 — Análise de dados
**Data:** 11/2025  

Este notebook é o segundo de três que analisam os ciclos políticos no Ceará nas últimas três décadas. A partir do desempenho partidário, a pesquisa deve mapear as legendas que cresceram e reduziram no Estado, e fornecer dados para uma reportagem que explique o que há por trás dessas oscilações.  

Para tanto, serão considerados os desempenhos partidários de prefeitos, deputados estaduais e federais. No caso dos mandatários municipais, eles revelam quem controla a política local e os recursos municipais. Já os deputados estaduais consolidam o apoio político ao Governo do Estado. E os deputados federais mostram como o Ceará se insere nas dinâmicas nacionais dos partidos.  

Neste segundo momento, os dados coletados a partir do [portal de Dados Abertos do Tribunal Superior Eleitoral](https://dadosabertos.tse.jus.br/) e padronizados serão analisados sob a perspectiva local e nacional.

Para a pesquisa, foram considerados os dados de Resultado das eleições municipais e gerais entre 1994 e 2024, totalizando 16 pleitos.

### Configuração inicial

Estrutura a análise conectando os arquivos processados anteriormente e definindo variáveis.

In [1]:
# Bibliotecas, caminhos, bloco de leituras e definições
import os
from pathlib import Path
import pandas as pd

pasta_base = r"C:\Python\eleicoes_ceara"
arquivo_dados = fr"{pasta_base}\dados_nacionais.csv"
arquivo_agrupado = fr"{pasta_base}\br_dados_processados\dados_deputados_agrupados.csv"
pasta_destino = fr"{pasta_base}\br_dados_processados"

CHUNK = 500_000
encoding_padrao = "utf-8"

In [2]:
# Leitura de arquivo processado
df = pd.concat(
    pd.read_csv(
        arquivo_dados,
        sep=";",
        dtype=str,
        chunksize=CHUNK,
        low_memory=False,
        encoding=encoding_padrao
    ),
    ignore_index=True
)

df

,Unnamed: 0,ANO_ELEICAO,SG_UF,SG_PARTIDO,DS_CARGO,DS_SIT_TOT_TURNO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_CANDIDATO.1,CD_MUNICIPIO,QT_VOTOS,NR_TURNO
0,0,1994,AC,PTC*,PRESIDENTE,NÃO ELEITO,28010003611655,36,CARLOS ANTONIO GOMES,CARLOS ANTONIO GOMES,1120,0,1
1,1,1994,AC,PRONA,PRESIDENTE,NÃO ELEITO,28010005611657,56,ENEAS FERREIRA CARNEIRO,ENEAS FERREIRA CARNEIRO,1120,0,1
2,2,1994,AC,PPR,PRESIDENTE,NÃO ELEITO,28010001111650,11,ESPERIDIAO AMIN HELOU FILHO,ESPERIDIAO AMIN HELOU FILHO,1120,0,1
3,3,1994,AC,PSDB,PRESIDENTE,ELEITO,28010004511656,45,FERNANDO HENRIQUE CARDOSO,FERNANDO HENRIQUE CARDOSO,1120,0,1
4,4,1994,AC,PSC,PRESIDENTE,NÃO ELEITO,28010002011654,20,HERNANI GOULART FORTUNA,HERNANI GOULART FORTUNA,1120,0,1


# Análise federal e estadual

Esta etapa analisa o comportamento partidário nos níveis estadual e federal, permitindo visualizar como os ciclos políticos locais se alinham — ou divergem — das tendências nacionais.

## Deputados estaduais e federais

In [8]:
# Filtragem
deputados = df[df["DS_CARGO"].isin(["DEPUTADO ESTADUAL", "DEPUTADO FEDERAL", "DEPUTADO DISTRITAL"])].copy()

### Funções

As funções a seguir automatizam o agrupamento e a classificação dos candidatos, identificando os eleitos e consolidando as informações por partido, estado e ano.

In [10]:
# Votação e processamento do status dos deputados
def gerar_base_deputados_agrupada(arquivo_entrada, arquivo_saida):
    prioridade = {
        "ELEITO": 4,
        "ELEITO POR QP": 3,
        "ELEITO POR MÉDIA": 2,
        "SUPLENTE": 1
    }
    inv = {
        4: "ELEITO",
        3: "ELEITO POR QP",
        2: "ELEITO POR MÉDIA",
        1: "SUPLENTE",
        0: "NÃO ELEITO/OUTRO"
    }

    dfs = []

    arquivo_entrada["QT_VOTOS"] = pd.to_numeric(arquivo_entrada["QT_VOTOS"], errors="coerce").fillna(0)
    arquivo_entrada["PRIORIDADE_STATUS"] = arquivo_entrada["DS_SIT_TOT_TURNO"].map(prioridade).fillna(0).astype(int)

    # Agrupa candidato e seleciona o status
    
    agg = (
        arquivo_entrada.groupby(
            ["ANO_ELEICAO", "SG_UF", "DS_CARGO", "SG_PARTIDO",
             "NR_CANDIDATO", "NM_CANDIDATO", "SQ_CANDIDATO"],
             as_index=False
             ).agg(
                 QT_VOTOS_TOTAL=("QT_VOTOS", "sum"),
                 PRIORIDADE_STATUS=("PRIORIDADE_STATUS", "max")
            )
        )

        # Nomeação do status
    agg["DS_SIT_TOT_TURNO"] = agg["PRIORIDADE_STATUS"].map(inv)
    dfs.append(agg)

    if not dfs:
        print("Nenhum dado processado.")
        return None

    # Montagem do df e exportação do arquivo
    df_final = (
        pd.concat(dfs, ignore_index=True)
          .groupby(["ANO_ELEICAO", "SG_UF", "DS_CARGO", "SG_PARTIDO",
                    "NR_CANDIDATO", "NM_CANDIDATO", "SQ_CANDIDATO", "DS_SIT_TOT_TURNO"],
                   as_index=False)["QT_VOTOS_TOTAL"].sum()
    )
    
    Path(arquivo_saida).parent.mkdir(parents=True, exist_ok=True)
    df_final.to_csv(arquivo_saida, sep=";", index=False, encoding=encoding_padrao)

    print(f"Base de deputados agregada salva em: {arquivo_saida}")
    return df_final

In [11]:
# Cálculo do "coeficiente" de desempenho partidário por estado
def calcular_coeficiente_deputados(df_deputados: pd.DataFrame) -> pd.DataFrame:
    if df_deputados.empty:
        return df_deputados.assign(total_estado=0, coeficiente=0.0)
    
    df_deputados = df_deputados.rename(columns={"DS_SIT_TOT_TURNO":"eleitos"})

    # Soma total de deputados eleitos por estado e ano
    totais = (
        df_deputados.groupby(["ANO_ELEICAO", "SG_UF", "DS_CARGO"], as_index=False)["eleitos"]
        .sum()
        .rename(columns={"eleitos": "total_estado"})
    )

    # Junta com os dados originais
    merged = df_deputados.merge(totais, on=["ANO_ELEICAO", "SG_UF", "DS_CARGO"], how="left")

    # Calcula o coeficiente (proporção)
    merged["coeficiente"] = (
        merged["eleitos"] / merged["total_estado"].where(merged["total_estado"] != 0, 1)
    )

    merged["coeficiente"] = round(merged["coeficiente"]*100, 2)

    return merged

### Geração de bases dos deputados

Cria as bases específicas de análise, organizando os dados em níveis com comparações entre eleições e unidades federativas.

In [12]:
df_deputados = gerar_base_deputados_agrupada(deputados, arquivo_agrupado)

Base de deputados agregada salva em: C:\Python\eleicoes_ceara\br_dados_processados\dados_deputados_agrupados.csv


In [13]:
# Filtra apenas os deputados eleitos
df_deputados_eleitos = df_deputados[df_deputados["DS_SIT_TOT_TURNO"].isin(["ELEITO", "ELEITO POR MÉDIA", "ELEITO POR QP"])]

# Exporta base com os eleitos
arquivo_eleitos = Path(pasta_destino) / "df_deputados_eleitos.csv"
df_deputados_eleitos.to_csv(arquivo_eleitos, sep=";", index=False, encoding=encoding_padrao)
print(f"Base de deputados eleitos salva em: {arquivo_eleitos}")

# Agrupa os eleitos por ano, UF, partido e cargo
dep_eleitos_agrupados = df_deputados_eleitos[["ANO_ELEICAO", "SG_UF", "SG_PARTIDO", "DS_SIT_TOT_TURNO", "DS_CARGO"]].groupby(["ANO_ELEICAO", "SG_UF", "SG_PARTIDO","DS_CARGO"]).count().reset_index()

# Exporta a tabela agrupada
arquivo_agrupado_eleitos = Path(pasta_destino) / "dep_eleitos_agrupados.csv"
dep_eleitos_agrupados.to_csv(arquivo_agrupado_eleitos, sep=";", index=False, encoding=encoding_padrao)

print(f"Base de eleitos salva em: {arquivo_agrupado_eleitos}")

Base de deputados eleitos salva em: C:\Python\eleicoes_ceara\br_dados_processados\df_deputados_eleitos.csv
Base de eleitos salva em: C:\Python\eleicoes_ceara\br_dados_processados\dep_eleitos_agrupados.csv


In [14]:
# Filtra apenas Ceará
df_deputados_eleitos_ce = dep_eleitos_agrupados[dep_eleitos_agrupados["SG_UF"] == "CE"].copy()

# Separa por tipo de cargo
df_estaduais_ce = df_deputados_eleitos_ce[df_deputados_eleitos_ce["DS_CARGO"] == "DEPUTADO ESTADUAL"].copy()
df_federais_ce = df_deputados_eleitos_ce[df_deputados_eleitos_ce["DS_CARGO"] == "DEPUTADO FEDERAL"].copy()

# Exporta bases separadas
arquivo_estaduais_ce = Path(pasta_destino) / "df_deputados_estaduais_eleitos_ce.csv"
arquivo_federais_ce = Path(pasta_destino) / "df_deputados_federais_eleitos_ce.csv"

df_estaduais_ce.to_csv(arquivo_estaduais_ce, sep=";", index=False, encoding=encoding_padrao)
df_federais_ce.to_csv(arquivo_federais_ce, sep=";", index=False, encoding=encoding_padrao)

print(f"Deputados estaduais eleitos do Ceará salvos em: {arquivo_estaduais_ce}")
print(f"Deputados federais eleitos do Ceará salvos em: {arquivo_federais_ce}")

Deputados estaduais eleitos do Ceará salvos em: C:\Python\eleicoes_ceara\br_dados_processados\df_deputados_estaduais_eleitos_ce.csv
Deputados federais eleitos do Ceará salvos em: C:\Python\eleicoes_ceara\br_dados_processados\df_deputados_federais_eleitos_ce.csv


### Cálculo do coeficiente partidário por estado

Expressa a fatia de representação conquistada por cada sigla nas eleições estaduais e federais.

In [15]:
coeficiente_dep = calcular_coeficiente_deputados(dep_eleitos_agrupados)
coeficiente_dep.to_csv(pasta_destino + "/coeficiente_dep.csv", index=False)

## Prefeitos

Aqui, o foco passa a ser o âmbito municipal, contabilizando as legendas que controlam o maior número de prefeituras.

In [16]:
# Filtragem
prefeitos = df[df["DS_CARGO"] == "PREFEITO"].copy()
prefeitos_eleitos = prefeitos[(prefeitos["DS_SIT_TOT_TURNO"] == "ELEITO")].copy()

### Função

A função isola os prefeitos eleitos e os agrupam por partido e estados, aglutinando a força político-partidária das legendas.

In [18]:
# Processa dados dos prefeitos eleitos
def gerar_base_prefeitos_agrupada(arquivo_entrada: str, arquivo_saida: str) -> pd.DataFrame:
    dfs = []
    # Seleção do último turno
    arquivo_entrada["NR_TURNO"] = pd.to_numeric(arquivo_entrada["NR_TURNO"], errors="coerce").fillna(1).astype(int)
    arquivo_entrada = arquivo_entrada.sort_values(
        ["ANO_ELEICAO", "SG_UF", "CD_MUNICIPIO", "NR_TURNO"],
        ascending=[True, True, True, False]
        )
    
    vencedores = arquivo_entrada.drop_duplicates(
        subset=["ANO_ELEICAO", "SG_UF", "CD_MUNICIPIO"], keep="first"
        )

    # Agrupamento dos prefeitos eleitos por ano, UF e partido
    agg = (
        vencedores.groupby(["ANO_ELEICAO", "SG_UF", "SG_PARTIDO"], as_index=False)
        .size()
        .rename(columns={"size": "eleitos"})
        )
    dfs.append(agg)

    if not dfs:
        print("Nenhum prefeito processado.")
        return pd.DataFrame(columns=["ANO_ELEICAO", "SG_UF", "SG_PARTIDO", "eleitos"])

    df_final = (
        pd.concat(dfs, ignore_index=True)
        .groupby(["ANO_ELEICAO", "SG_UF", "SG_PARTIDO"], as_index=False)["eleitos"]
        .sum()
    )

    Path(arquivo_saida).parent.mkdir(parents=True, exist_ok=True)
    df_final.to_csv(arquivo_saida, sep=";", index=False, encoding=encoding_padrao)

    print(f"Base de prefeitos eleitos salva em: {arquivo_saida}")

    return df_final

### Cálculo do coeficiente partidário por estado

Aplica aos prefeitos a mesma lógica dos coeficientes legislativos, revelando a proporção de influência da base municipal dos partidos nos ciclos eleitorais.

In [19]:
# Cálculo do "coeficiente" de desempenho partidário por estado
def calcular_coeficiente_prefeitos(df_prefeitos: pd.DataFrame) -> pd.DataFrame:
    if df_prefeitos.empty:
        return df_prefeitos.assign(total_estado=0, coeficiente=0.0)

    # Soma de prefeitos eleitos por estado e ano
    totais = (
        df_prefeitos.groupby(["ANO_ELEICAO", "SG_UF"], as_index=False)["eleitos"]
        .sum()
        .rename(columns={"eleitos": "total_estado"})
    )

    merged = df_prefeitos.merge(totais, on=["ANO_ELEICAO", "SG_UF"], how="left")

    # Cálculo da proporção
    merged["coeficiente"] = (
        merged["eleitos"] / merged["total_estado"].where(merged["total_estado"] != 0, 1)
    )

    merged["coeficiente"] = round(merged["coeficiente"]*100, 2)

    return merged

### Geração da base dos prefeitos

Gera os arquivos finais que consolidam os prefeitos eleitos e seus coeficientes de representação estadual.

In [20]:
# Gera e salva a base de prefeitos eleitos
arquivo_prefeitos = Path(pasta_destino) / "prefeitos_eleitos.csv"
df_prefeitos = gerar_base_prefeitos_agrupada(prefeitos_eleitos, arquivo_prefeitos)

# Gera e salva o coeficiente
coeficiente_prefeitos = calcular_coeficiente_prefeitos(df_prefeitos)
coeficiente_prefeitos.to_csv(pasta_destino + "/coeficiente_prefeitos.csv", index=False)

Base de prefeitos eleitos salva em: C:\Python\eleicoes_ceara\br_dados_processados\prefeitos_eleitos.csv


In [21]:
# Filtra apenas prefeitos eleitos do Ceará
prefeitos_eleitos_ce = df_prefeitos [df_prefeitos ["SG_UF"] == "CE"].copy()

# Exporta apenas os prefeitos eleitos do Ceará
df_sg_prefeitos_eleitos_ce = Path(pasta_destino) / "prefeitos_eleitos_ce.csv"
prefeitos_eleitos_ce.to_csv(df_sg_prefeitos_eleitos_ce, sep=";", index=False, encoding=encoding_padrao)

print(f"Base de prefeitos eleitos do Ceará salva em: {df_sg_prefeitos_eleitos_ce}")

Base de prefeitos eleitos do Ceará salva em: C:\Python\eleicoes_ceara\br_dados_processados\prefeitos_eleitos_ce.csv


# [DESCONSIDERAR] Análise municipal

Nesta etapa, chega o nível mais granular da pesquisa, com a análise municipal. Ela permite observar onde cada partido se fortalece e como os padrões locais refletem (ou destoam) das tendências estaduais.

## Deputados estaduais e federais

Analisa a distribuição dos votos de deputados por município e identifica os cinco partidos mais votados em cada cidade e pleito, revelando bases eleitorais.

In [15]:
#Filtragem
deputados_ce = deputados[deputados["SG_UF"] == "CE"]

In [16]:
# Quantidade de votos em cada município recebidos por cada partido
partidos_deputados_municipios_ce = deputados_ce[["ANO_ELEICAO", "SG_PARTIDO", "DS_CARGO", "CD_MUNICIPIO", "QT_VOTOS"]].groupby(
    ["ANO_ELEICAO", "CD_MUNICIPIO", "SG_PARTIDO", "DS_CARGO"]
    )["QT_VOTOS"].sum().reset_index().sort_values(["QT_VOTOS"], ascending=False)

In [17]:
partidos_deputados_municipios_ce["QT_VOTOS"] = pd.to_numeric(
    partidos_deputados_municipios_ce["QT_VOTOS"], errors="coerce"
)

# Agrupa por ano, município, cargo e partido, somando os votos
votos_agrupados = (
    partidos_deputados_municipios_ce
    .groupby(["ANO_ELEICAO", "CD_MUNICIPIO", "DS_CARGO", "SG_PARTIDO"], as_index=False)["QT_VOTOS"]
    .sum()
)

# Ordena dentro de cada grupo e seleciona os 5 mais votados
top5_por_municipio_ano_cargo = (
    votos_agrupados
    .sort_values(["ANO_ELEICAO", "CD_MUNICIPIO", "DS_CARGO", "QT_VOTOS"], ascending=[True, True, True, False])
    .groupby(["ANO_ELEICAO", "CD_MUNICIPIO", "DS_CARGO"])
    .head(5)
)

In [18]:
# Gera a base dos partidos mais votados nos municípios
top5_por_municipio_ano_cargo.to_csv(pasta_destino + "/top5_por_municipio_ano_deputados.csv", index=False)

## Prefeitos

Compila o partido dos prefeitos vencedores em cada município e eleição, criando um panorama histórico do controle partidário municipal no Ceará.

In [19]:
# Partido eleito em cada município do Ceará em cada eleição
prefeitos_eleitos_ce = prefeitos_eleitos[prefeitos_eleitos["SG_UF"] == "CE"]
prefeitos_eleitos_ce = prefeitos_eleitos_ce[["ANO_ELEICAO", "SG_PARTIDO", "NM_CANDIDATO", "CD_MUNICIPIO"]]

prefeitos_eleitos_ce = prefeitos_eleitos_ce.sort_values(
    ["ANO_ELEICAO", "CD_MUNICIPIO"],
    ascending=[True, True]
    )
    
prefeitos_vencedores_ce = prefeitos_eleitos_ce.drop_duplicates(
    subset=["ANO_ELEICAO", "CD_MUNICIPIO"], keep="first"
    )


In [20]:
# Gera a base com o partido dos prefeitos eleitos
prefeitos_vencedores_ce.to_csv(pasta_destino + "/prefeitos_vencedores_ce.csv", index=False)

# Coeficientes nacionais

Amarra as escalas de análise — municipal, estadual e nacional — comparando o desempenho dos partidos no Ceará com o restante do país ao longo das décadas.

### Deputados federais e estaduais

Calcula os coeficientes de representação dos partidos nas eleições legislativas, permitindo observar padrões de crescimento e retração.

In [22]:
# Filtragem do coeficiente de deputados do CE
coeficientes_deputados = pd.read_csv(pasta_destino+"/coeficiente_dep.csv")
df_coeficientes_deputados_ce = coeficientes_deputados[coeficientes_deputados["SG_UF"] =="CE"]

In [23]:
# Agrupamento nacional dos deputados
coeficientes_deputados = coeficientes_deputados[["ANO_ELEICAO", "SG_PARTIDO", "DS_CARGO", "eleitos"]]
coeficientes_deputados = coeficientes_deputados.groupby(["ANO_ELEICAO", "SG_PARTIDO", "DS_CARGO"]).sum().reset_index()

In [24]:
total_eleitos_deputados = coeficientes_deputados[["ANO_ELEICAO", "DS_CARGO", "eleitos"]].groupby(["ANO_ELEICAO", "DS_CARGO"]).sum().reset_index()

In [25]:
# Cálculo do coeficiente nacional de deputados
df_coef_deputados = coeficientes_deputados.merge(total_eleitos_deputados, on=["ANO_ELEICAO", "DS_CARGO"]).rename(
    columns={"eleitos_x": "eleitos_partido", "eleitos_y": "total"})
df_coef_deputados["coeficiente"] = round(df_coef_deputados["eleitos_partido"]/df_coef_deputados["total"]*100,2)
df_coef_deputados.rename(columns={"eleitos_partido": "eleitos_partido_BR",
                                  "total": "total_BR",
                                  "coeficiente": "coeficiente_BR"
                                  },inplace=True)
df_coef_deputados.head()

,ANO_ELEICAO,SG_PARTIDO,DS_CARGO,eleitos_partido_BR,total_BR,coeficiente_BR
0,1994,CIDADANIA*,DEPUTADO ESTADUAL,3,1045,0.29
1,1994,CIDADANIA*,DEPUTADO FEDERAL,2,513,0.39
2,1994,DEM*,DEPUTADO ESTADUAL,157,1045,15.02
3,1994,DEM*,DEPUTADO FEDERAL,89,513,17.35
4,1994,MDB*,DEPUTADO ESTADUAL,205,1045,19.62


In [26]:
# coeficiente de desempenho partidário do CE
df_coeficientes_deputados_ce = df_coeficientes_deputados_ce[["ANO_ELEICAO", "SG_UF", "SG_PARTIDO", "DS_CARGO", "eleitos", "total_estado"]]
df_coeficientes_deputados_ce = df_coeficientes_deputados_ce.groupby(["ANO_ELEICAO", "SG_UF", "SG_PARTIDO", "DS_CARGO"]).sum().reset_index()
df_coeficientes_deputados_ce["coeficiente"] = round(df_coeficientes_deputados_ce["eleitos"]/df_coeficientes_deputados_ce["total_estado"]*100,2)
df_coeficientes_deputados_ce.rename(columns={"eleitos": "eleitos_partido_CE",
                                  "total_estado": "total_CE",
                                  "coeficiente": "coeficiente_CE"
                                  },inplace=True)
df_coeficientes_deputados_ce.head()

,ANO_ELEICAO,SG_UF,SG_PARTIDO,DS_CARGO,eleitos_partido_CE,total_CE,coeficiente_CE
0,1994,CE,DEM*,DEPUTADO ESTADUAL,2,46,4.35
1,1994,CE,DEM*,DEPUTADO FEDERAL,2,22,9.09
2,1994,CE,MDB*,DEPUTADO ESTADUAL,5,46,10.87
3,1994,CE,MDB*,DEPUTADO FEDERAL,5,22,22.73
4,1994,CE,PC DO B,DEPUTADO FEDERAL,1,22,4.55


In [27]:
# Unifica as tabelas do Brasil e do Ceará para os coeficientes de eleição para deputado por partido
coeficientes_BR_CE = df_coef_deputados.merge(df_coeficientes_deputados_ce, on=["ANO_ELEICAO", "SG_PARTIDO", "DS_CARGO"], how="left")
coeficientes_BR_CE.drop(columns="SG_UF", inplace=True)

In [29]:
# Padroniza os valores nulos do merge anterior
coeficientes_BR_CE["eleitos_partido_CE"] = coeficientes_BR_CE["eleitos_partido_CE"].fillna(0).astype(int)
coeficientes_BR_CE["coeficiente_CE"] = coeficientes_BR_CE["coeficiente_CE"].fillna(0)

# Cria um dicionário com o total de vagas (total_CE) por ano e cargo
totais_ce = (
    coeficientes_BR_CE
    .dropna(subset=["total_CE"])
    .groupby(["ANO_ELEICAO", "DS_CARGO"])["total_CE"]
    .first()
    .to_dict()
)

# Preenche os valores ausentes com base no ano
coeficientes_BR_CE["total_CE"] = coeficientes_BR_CE.apply(
    lambda row: totais_ce.get((row["ANO_ELEICAO"], row["DS_CARGO"]), row["total_CE"]),
    axis=1
)

# Converte para numérico e substitui valores ausentes por 0
coeficientes_BR_CE["total_CE"] = pd.to_numeric(coeficientes_BR_CE["total_CE"], errors="coerce").fillna(0).astype(int)
coeficientes_BR_CE.to_csv(pasta_destino + "/coeficientes_deputados_BR_CE.csv", index=False)
coeficientes_BR_CE.head()

,ANO_ELEICAO,SG_PARTIDO,DS_CARGO,eleitos_partido_BR,total_BR,coeficiente_BR,eleitos_partido_CE,total_CE,coeficiente_CE
0,1994,CIDADANIA*,DEPUTADO ESTADUAL,3,1045,0.29,0,46,0.00
1,1994,CIDADANIA*,DEPUTADO FEDERAL,2,513,0.39,0,22,0.00
2,1994,DEM*,DEPUTADO ESTADUAL,157,1045,15.02,2,46,4.35
3,1994,DEM*,DEPUTADO FEDERAL,89,513,17.35,2,22,9.09
4,1994,MDB*,DEPUTADO ESTADUAL,205,1045,19.62,5,46,10.87


In [30]:
coeficientes_dep_federal_BR_CE = coeficientes_BR_CE[coeficientes_BR_CE["DS_CARGO"] == "DEPUTADO FEDERAL"]
coeficientes_dep_federal_BR_CE.to_csv(pasta_destino + "/coeficientes_dep_federal_BR_CE.csv", index=False)


coeficientes_dep_estadual_BR_CE = coeficientes_BR_CE[coeficientes_BR_CE["DS_CARGO"] == "DEPUTADO ESTADUAL"]
coeficientes_dep_estadual_BR_CE.to_csv(pasta_destino + "/coeficientes_dep_estadual_BR_CE.csv", index=False)

### Prefeitos

Compara o peso dos partidos no total de prefeituras brasileiras e cearenses.

In [31]:
# Filtragem do coeficiente de prefeitos do CE
coeficientes_prefeitos = pd.read_csv(pasta_destino+"/coeficiente_prefeitos.csv")
coeficientes_prefeitos.drop(columns="coeficiente", inplace=True)
coeficientes_prefeitos_ce = coeficientes_prefeitos[coeficientes_prefeitos["SG_UF"] =="CE"]

In [32]:
# Agrupamento nacional dos prefeitos eleitos
prefeitos_eleitos_partido = coeficientes_prefeitos[["ANO_ELEICAO", "SG_PARTIDO", "eleitos"]]
prefeitos_eleitos_partido = prefeitos_eleitos_partido.groupby(["ANO_ELEICAO", "SG_PARTIDO"]).sum().reset_index()
prefeitos_eleitos_partido

array(['AVANTE*', 'CIDADANIA*', 'DC*', 'DEM*', 'MDB*', 'MOBILIZA*', 'PAN',
       'PC DO B', 'PDT', 'PHS*', 'PL*', 'PP*', 'PRP', 'PRTB*', 'PSB',
       'PSC', 'PSD', 'PSDB', 'PSL', 'PST', 'PT', 'PTB', 'PTC*', 'PTN',
       'PV', 'AGIR*', 'PRONA', 'REPUBLICANOS*', 'PPL', 'PSOL', 'PATRI',
       'PATRIOTA*', 'PMB', 'PODE', 'PROS', 'REDE', 'SOLIDARIEDADE*',
       'NOVO', 'UNIÃO', 'PRD'], dtype=object)

In [33]:
total_eleitos_prefeitos = coeficientes_prefeitos[["ANO_ELEICAO", "eleitos"]].groupby(["ANO_ELEICAO"]).sum().reset_index()

In [34]:
# Cálculo do coeficiente nacional de prefeitos
df_coef_prefeitos = prefeitos_eleitos_partido.merge(total_eleitos_prefeitos, on=["ANO_ELEICAO"]).rename(
    columns={"eleitos_x": "eleitos_partido", "eleitos_y": "total"})
df_coef_prefeitos["coeficiente"] = round(df_coef_prefeitos["eleitos_partido"]/df_coef_prefeitos["total"]*100,2)
df_coef_prefeitos.rename(columns={"eleitos_partido": "eleitos_partido_BR",
                                  "total": "total_BR",
                                  "coeficiente": "coeficiente_BR"
                                  },inplace=True)
df_coef_prefeitos.head()

,ANO_ELEICAO,SG_PARTIDO,eleitos_partido_BR,total_BR,coeficiente_BR
0,2000,AVANTE*,6,5555,0.11
1,2000,CIDADANIA*,166,5555,2.99
2,2000,DC*,8,5555,0.14
3,2000,DEM*,1026,5555,18.47
4,2000,MDB*,1256,5555,22.61


In [35]:
# coeficiente de desempenho partidário do CE
coeficientes_prefeitos_ce[["ANO_ELEICAO", "SG_UF", "SG_PARTIDO", "eleitos", "total_estado"]]
coeficientes_prefeitos_ce = coeficientes_prefeitos_ce.groupby(["ANO_ELEICAO", "SG_UF", "SG_PARTIDO"]).sum().reset_index()
coeficientes_prefeitos_ce["coeficiente"] = round(coeficientes_prefeitos_ce["eleitos"]/coeficientes_prefeitos_ce["total_estado"]*100,2)
coeficientes_prefeitos_ce.rename(columns={"eleitos": "eleitos_partido_CE",
                                  "total_estado": "total_CE",
                                  "coeficiente": "coeficiente_CE"
                                  },inplace=True)
coeficientes_prefeitos_ce

,ANO_ELEICAO,SG_UF,SG_PARTIDO,eleitos_partido_CE,total_CE,coeficiente_CE
0,2000,CE,CIDADANIA*,17,184,9.24
1,2000,CE,DEM*,11,184,5.98
2,2000,CE,MDB*,12,184,6.52
3,2000,CE,PL*,1,184,0.54
4,2000,CE,PP*,10,184,5.43
...,...,...,...,...,...,...
109,2024,CE,PSDB,2,182,1.10
110,2024,CE,PT,47,182,25.82
111,2024,CE,REPUBLICANOS*,14,182,7.69
112,2024,CE,SOLIDARIEDADE*,1,182,0.55


In [36]:
# Unifica as tabelas do Brasil e do Ceará para os coeficientes de eleição dos prefeitos por partido
coeficientes_prefeitos_BR_CE = df_coef_prefeitos.merge(coeficientes_prefeitos_ce, on=["ANO_ELEICAO", "SG_PARTIDO"], how="left")
coeficientes_prefeitos_BR_CE.drop(columns="SG_UF", inplace=True)

In [38]:
coeficientes_prefeitos_BR_CE

,ANO_ELEICAO,SG_PARTIDO,eleitos_partido_BR,total_BR,coeficiente_BR,eleitos_partido_CE,total_CE,coeficiente_CE
0,2000,AVANTE*,6,5555,0.11,NaN,NaN,NaN
1,2000,CIDADANIA*,166,5555,2.99,17.0,184.0,9.24
2,2000,DC*,8,5555,0.14,NaN,NaN,NaN
3,2000,DEM*,1026,5555,18.47,11.0,184.0,5.98
4,2000,MDB*,1256,5555,22.61,12.0,184.0,6.52
...,...,...,...,...,...,...,...,...
180,2024,PV,13,5549,0.23,NaN,NaN,NaN
181,2024,REDE,4,5549,0.07,NaN,NaN,NaN
182,2024,REPUBLICANOS*,437,5549,7.88,14.0,182.0,7.69
183,2024,SOLIDARIEDADE*,64,5549,1.15,1.0,182.0,0.55


In [39]:
# Padroniza os valores nulos do merge anterior
coeficientes_prefeitos_BR_CE["eleitos_partido_CE"] = (coeficientes_prefeitos_BR_CE["eleitos_partido_CE"].fillna(0).astype(int))
coeficientes_prefeitos_BR_CE["coeficiente_CE"] = (coeficientes_prefeitos_BR_CE["coeficiente_CE"].fillna(0))

# Cria dicionário com total_CE conhecido por ano
totais_ce_prefeitos = (
    coeficientes_prefeitos_BR_CE
    .dropna(subset=["total_CE"])
    .groupby("ANO_ELEICAO")["total_CE"]
    .first()
    .to_dict()
)

# Preenche os valores ausentes com base no ano
coeficientes_prefeitos_BR_CE["total_CE"] = coeficientes_prefeitos_BR_CE.apply(
    lambda row: totais_ce_prefeitos.get(row["ANO_ELEICAO"], row["total_CE"]),
    axis=1
)

# Converte para numérico e substitui valores ausentes por 0
coeficientes_prefeitos_BR_CE["total_CE"] = (pd.to_numeric(coeficientes_prefeitos_BR_CE["total_CE"], errors="coerce").fillna(0).astype(int))
#coeficientes_prefeitos_BR_CE["diferenca_CE_BR"] = coeficientes_prefeitos_BR_CE["coeficiente_CE"]-coeficientes_prefeitos_BR_CE["coeficiente_BR"]

coeficientes_prefeitos_BR_CE.to_csv(pasta_destino + "/coeficientes_prefeitos_BR_CE.csv", index=False)
coeficientes_prefeitos_BR_CE.head()

,ANO_ELEICAO,SG_PARTIDO,eleitos_partido_BR,total_BR,coeficiente_BR,eleitos_partido_CE,total_CE,coeficiente_CE
0,2000,AVANTE*,6,5555,0.11,0,184,0.00
1,2000,CIDADANIA*,166,5555,2.99,17,184,9.24
2,2000,DC*,8,5555,0.14,0,184,0.00
3,2000,DEM*,1026,5555,18.47,11,184,5.98
4,2000,MDB*,1256,5555,22.61,12,184,6.52
